In [1]:
import datetime
import numpy
import netCDF4
from osgeo import gdal, osr
import ogr
import os
import pandas as pd

#example
#start = '2015-01-01'
#end = '2015-03-23'
#sd = dt.strptime(start,'%Y-%m-%d')
#ed = dt.strptime(end,'%Y-%m-%d')
#delta = ed - sd
#for i in range(delta.days+1):
#    dict[sd + td(days=i)] = <something>

reqStart = '1960-01-01'
reqEnd = '1960-12-31'

#inputGeoCoordsFile = r'P:\projects\SILO\Wes\MNDB_geocoords.csv'
#outputDir = r'P:\projects\SILO\Wes\testOut'
#siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

inputGeoCoordsFile = r'C:\DDrive\Maria\CatchModCSVs\WT_RC8_30m_climcoords.csv'
outputDir = r'C:\DDrive\Maria\ClimateOuts'
siloNetCDFDir = r'C:\DDrive\Maria\netCDFs\rainfall'

varNameInSilo = 'daily_rain'
varForColName = 'rainfall'

#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'

In [2]:
#Read in GeoCoords
GeoCoordsVals = {}

geoCoordsDF = pd.read_csv(inputGeoCoordsFile)
#for index, row in geoCoordsDF.iterrows():
#    if not row[catGrpName] in GeoCoordsVals:
#        #print('Adding: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
#        GeoCoordsVals[row[catGrpName]] = row[catFileCatName]

allCats = geoCoordsDF[catGrpName].unique().tolist()
colNames = [varForColName + ' for ' + cat for cat in allCats]
print(len(allCats))

517


In [3]:

sd = datetime.datetime.strptime(reqStart,'%Y-%m-%d')
ed = datetime.datetime.strptime(reqEnd,'%Y-%m-%d')
##delta = ed - sd

sYear = sd.year
eYear = ed.year

allYears = []
for y in range(sYear, eYear + 1):
    if not y in allYears:
        allYears.append(y)

#date_today = datetime.now()
#days = pd.date_range(date_today, date_today + timedelta(7), freq='D')
date_rng = pd.date_range(sd, ed, freq='D')
#print(str(type(days[0])))
daysAsList = date_rng.tolist()
#mainDF = pd.DataFrame({'Date': days})
mainDF = pd.DataFrame(date_rng, columns=['tempDate'])
#mainDF = mainDF.set_index('Date')

#Getting date as index????
mainDF['Date'] = pd.to_datetime(mainDF['tempDate'])
mainDF = mainDF.set_index('Date')
mainDF.drop(['tempDate'], axis=1, inplace=True)


for col in colNames:
    mainDF[col] = ''


mainDF.head()
#print(mainDF)

,rainfall for SC #4,rainfall for SC #35,rainfall for SC #1,rainfall for SC #5,rainfall for SC #34,rainfall for SC #90488,rainfall for SC #2,rainfall for SC #3,rainfall for SC #9,rainfall for SC #33,...,rainfall for SC #381,rainfall for SC #379,rainfall for SC #380,rainfall for SC #469,rainfall for SC #376,rainfall for SC #386,rainfall for SC #377,rainfall for SC #378,rainfall for SC #387,rainfall for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
1960-01-01,,,,,,,,,,,...,,,,,,,,,,
1960-01-02,,,,,,,,,,,...,,,,,,,,,,
1960-01-03,,,,,,,,,,,...,,,,,,,,,,
1960-01-04,,,,,,,,,,,...,,,,,,,,,,
1960-01-05,,,,,,,,,,,...,,,,,,,,,,


In [9]:
#load the data
print(allYears)

print('started at: ' + str(datetime.datetime.now()))

for thisYear in allYears:
    
    siloFile = os.path.join(siloNetCDFDir, str(thisYear) + '.' + varNameInSilo + '.nc')
    print('Processing: ' + siloFile)
    #print(type(thisYear))
    
    #Need the gdal version of silo to give us a few params, but will close of netCDF version pronto...
    siloDS = gdal.Open(siloFile)
    
    ##Need this to calc pixel size as well as work out order of y-axis
    netCDFData = netCDF4.Dataset(siloFile, 'r')

    #Grab the info we need to build GeoTransform object
    #Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array
    
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    first_lat = lats[:1]
    last_lat = lats[len(lats)-1:]
    #If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...
    
    first_lon = lons[:1]
    last_lon = lons[len(lons)-1:]
    
    colCount = siloDS.RasterXSize
    rowCount = siloDS.RasterYSize
    
    pixWidth = (last_lon - first_lon) / (siloDS.RasterXSize - 1)
    pixHeight = (last_lat - first_lat) / (siloDS.RasterYSize - 1)

    if first_lat < last_lat:
        #adjust for our use
        pixHeight *= -1
    
    
    siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
    siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre
    
    netCDFData.close()
    
    fakeSiloGT = [siloLeft, pixWidth, 0, siloTop, 0, pixHeight]
    
    
    
    #Jan 1
    siloDStart = datetime.datetime(year=thisYear, month=1, day=1)
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    for band in range( siloDS.RasterCount ):
        
        dateAdd = band
        dateNow = siloDStart + datetime.timedelta(days=dateAdd)
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        
        
        if not dateNow in daysAsList:
            continue

        #dateTimeStampKey = pd.Timestamp(dateNow, freq='D')#
        #dateTimeStampKey = datetime.datetime.strptime(dateForFileName,'%Y%m%d')
        dateTimeStampKey = str(dateNow.year) + '-' + str(dateNow.month).zfill(2) + '-' + str(dateNow.day).zfill(2)
        #print('Here: ' + str(dateTimeStampKey))
        #print('Here: ' + str(dateNow))
        
        band += 1
        #print("[ GETTING BAND ]: ", band)
        srcband = siloDS.GetRasterBand(band)
        
        if srcband is None:
            continue
            
        scaleFactor = 1.0
        
        #sf = srcband.GetMetadataItem(varNameInSilo + '_scale_factor')
        #nd = srcband.GetMetadataItem(varNameInSilo + '__FillValue')
        sf = srcband.GetMetadataItem('scale_factor')
        nd = srcband.GetMetadataItem('_FillValue')
        nd2 = srcband.GetNoDataValue()
        
        #if band == 1:
            #print('SRCBAND is : ' + str(type(srcband)))
            #print('ND: ' + str(nd) + ' and ND2: ' + str(nd2))
        
        #print(sf)
        
        scaleFactor = float(sf)
        rawNoData = float(nd)
        
        siloArray = srcband.ReadAsArray()
        #just make sure there's no gaps? Not necessary?
        #siloMask = numpy.ma.masked_where(siloArray == nd, siloArray)
        
        #print(dateForFileName + ': ' + str(siloMask[500,500] * scaleFactor))
        
        for thisCat in allCats:
            
            colName = varForColName + ' for ' + thisCat
            catTodaySum = 0.0
            catTodayCount = 0.0
            
            catDF = geoCoordsDF.loc[geoCoordsDF[catGrpName] == thisCat]
            
            for index, row in catDF.iterrows():
                
                longVal = float(row[longName])
                latVal = float(row[latName])
                countVal = int(row[countName])
                
                geoXcell = int((longVal - fakeSiloGT[0]) / fakeSiloGT[1]) #x pixel
                geoYcell = int((latVal - fakeSiloGT[3]) / fakeSiloGT[5]) #y pixel
                
                
                catTodaySum += (siloArray[geoYcell,geoXcell] * scaleFactor) * countVal
                catTodayCount += countVal
        
        
            #Calc average
            #avVal = 0
            if catTodayCount > 0:
                avVal = catTodaySum / catTodayCount
            #print('Today av: ' + str(avVal))
            #update mainDF
            mainDF.loc[dateTimeStampKey][colName] = avVal
        
#mainDF.head()
#print('finished')
print('finished at: ' + str(datetime.datetime.now()))

[1960]
started at: 2019-11-04 12:11:50.919402
Processing: C:\DDrive\Maria\netCDFs\rainfall\1960.daily_rain.nc
finished at: 2019-11-04 13:01:42.189046


In [7]:
mainDF.head(20)

,rainfall for SC #4,rainfall for SC #35,rainfall for SC #1,rainfall for SC #5,rainfall for SC #34,rainfall for SC #90488,rainfall for SC #2,rainfall for SC #3,rainfall for SC #9,rainfall for SC #33,...,rainfall for SC #381,rainfall for SC #379,rainfall for SC #380,rainfall for SC #469,rainfall for SC #376,rainfall for SC #386,rainfall for SC #377,rainfall for SC #378,rainfall for SC #387,rainfall for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
1960-01-01,,,,,,,,,,,...,,,,,,,,,,
1960-01-02,,,,,,,,,,,...,,,,,,,,,,
1960-01-03,,,,,,,,,,,...,,,,,,,,,,
1960-01-04,,,,,,,,,,,...,,,,,,,,,,
1960-01-05,,,,,,,,,,,...,,,,,,,,,,
1960-01-06,,,,,,,,,,,...,,,,,,,,,,
1960-01-07,,,,,,,,,,,...,,,,,,,,,,
1960-01-08,,,,,,,,,,,...,,,,,,,,,,
1960-01-09,,,,,,,,,,,...,,,,,,,,,,
